In [3]:
def generate_buy_above_ema50_signal(df):
    long_signal_condition = (df['open'] > df['ema50']) & (df['date'].dt.weekday == 2)
    df.loc[long_signal_condition, 'signal'] = True
    df.loc[long_signal_condition, 'regular_investment'] = 100
    return df

In [4]:
def generate_buy_under_ema50_signal(df):
    long_signal_condition = (df['open'] < df['ema50']) & (df['date'].dt.weekday == 2)
    df.loc[long_signal_condition, 'signal'] = True
    df.loc[long_signal_condition, 'regular_investment'] = 100
    return df

In [6]:
def generate_buy_every_week_signal(df):
    tuesday_signal_condition = (df['date'].dt.weekday == 2)
    df.loc[tuesday_signal_condition, 'signal'] = True
    df.loc[tuesday_signal_condition, 'regular_investment'] = 100
    return df

In [7]:
def generate_buy_signal_dynamic_investment(df):
    long_signal_condition = (df['open'] < df['ema50']) & (df['date'].dt.weekday == 2)
    long_signal_condition2 = (df['open'] > df['ema50']) & (df['date'].dt.weekday == 2)
    df.loc[long_signal_condition, 'signal'] = True
    df.loc[long_signal_condition, 'regular_investment'] = 75
    df.loc[long_signal_condition2, 'signal'] = True
    df.loc[long_signal_condition2, 'regular_investment'] = 25
    return df

In [2]:
def generate_buy_signal_Volatility_adjusted(df):
    dfvix= yf.download('^VIX', start='2000-11-30', end='2024-1-29', interval='1d', progress=False)
    dfvix=dfvix.reset_index()
    dfvix.columns = [x.lower() for x in dfvix.columns]
    dfvix.rename(columns={'open': 'openvix'}, inplace=True)
    dfmedian = dfvix['openvix'].median()
    df = pd.merge(df, dfvix[['date','openvix']], on='date')

    long_signal_condition = (df['openvix'] < dfmedian) & (df['date'].dt.weekday == 2)
    long_signal_condition2 = (df['openvix'] > dfmedian) & (df['date'].dt.weekday == 2)
    df.loc[long_signal_condition, 'signal'] = True
    df.loc[long_signal_condition, 'regular_investment'] = 75
    df.loc[long_signal_condition2, 'signal'] = True
    df.loc[long_signal_condition2, 'regular_investment'] = 25
    return df

In [ ]:
def calculate_seasonality(df, column_name):
    """
    Calculate seasonality for a DataFrame with 'Date' and the specified column.
    Parameters:
    - df (pd.DataFrame): Input DataFrame.
    - column_name (str): Name of the column for which to calculate seasonality.
    Returns:
    - pd.DataFrame: DataFrame with calculated seasonality.
    """
    # Copy the input DataFrame to avoid modifying the original
    df_copy = df.copy()
    # If 'Date' is not in columns, set it as the index
    if 'date' not in df_copy.columns:
        df_copy= df_copy.reset_index()
    # Convert 'Date' column to datetime if it's not already
    df_copy['date'] = pd.to_datetime(df_copy['date'])
    # Extract month from the 'date' column and create a new column 'Month'
    df_copy['Month'] = df_copy['date'].dt.month
    # Group by month and calculate the mean
    seasonality_data = df_copy.groupby('Month')[column_name].mean().reset_index()
    return seasonality_data.round(2)

In [ ]:
def generate_buy_signal_seasonal_DCA(df):
    df['regular_investment']= ''
    df['Change%'] = (df['open'].pct_change(fill_method=None) * 100).round(2)
    seasonality = calculate_seasonality(df, 'Change%')
    df['month']= df['date'].dt.month
    for i in range(len(df)):
        if (seasonality.loc[seasonality['Month'] == df['month'][i], 'Change%'] < 0).all() and df['date'][i].weekday() == 2 :
            df['signal'][i] = True
            df['regular_investment'][i] = 25
        if (seasonality.loc[seasonality['Month'] == df['month'][i], 'Change%'] >= 0).all() and df['date'][i].weekday() == 2 :
            df['signal'][i] = True
            df['regular_investment'][i] = 75
    return df

In [ ]:
#def generate_buy_signal_sentiment_DCA(df):